In [3]:
import requests
import json
from streamlit.connections import ExperimentalBaseConnection

class OpenWeatherMapConnection(ExperimentalBaseConnection[requests.Session]):
    def __init__(self, *args, connection_name, **kwargs):
        super().__init__(*args, connection_name=connection_name, **kwargs)
        self._resource = self._connect()

    def _connect(self) -> requests.Session:
        return requests.Session()

    def cursor(self):
        return self._resource

    def get_weather(self, city_name):
        with self._resource as s:
            url = 'http://api.openweathermap.org/data/2.5/weather'
            params = {
                'q': city_name,
                'appid': 'fe952f9b69a7b12dd4e42d3f2a55f6f4',
                'units': 'metric'  # Use 'imperial' for Fahrenheit
            }

            response = s.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            weather_description = data['weather'][0]['description']
            temperature = data['main']['temp']
            humidity = data['main']['humidity']

            return {
                "weather_description": weather_description,
                "temperature": temperature,
                "humidity": humidity
            }
        else:
            raise ValueError("Failed to retrieve weather information.")

# Usage example:
if __name__ == "__main__":
    api_key = 'fe952f9b69a7b12dd4e42d3f2a55f6f4'  # Replace with your actual API key
    city_name = 'London'  # Replace with the desired city name

    connection = OpenWeatherMapConnection(connection_name="OpenWeatherMap")
    connection._resource.headers.update({'appid': api_key})

    try:
        weather_data = connection.get_weather(city_name)
        print(f"Weather in {city_name}: {weather_data['weather_description']}")
        print(f"Temperature: {weather_data['temperature']} °C")
        print(f"Humidity: {weather_data['humidity']}%")
    except ValueError as e:
        print(e)

Weather in London: few clouds
Temperature: 22.95 °C
Humidity: 61%
